<a href="https://colab.research.google.com/github/Likelipop/DQL_Trading_based/blob/main/new_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install vnstock3

In [3]:
!pip install stable-baselines3 gymnasium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
import pandas as pd
from vnstock import Vnstock


    Khi tiếp tục sử dụng Vnstock3, bạn xác nhận rằng bạn đã đọc, hiểu và đồng ý với Chính sách quyền riêng tư và Điều khoản, điều kiện về giấy phép sử dụng Vnstock3.

    Chi tiết:

    - Giấy phép sử dụng phần mềm: https://vnstocks.com/docs/tai-lieu/giay-phep-su-dung
    - Chính sách quyền riêng tư: https://vnstocks.com/docs/tai-lieu/chinh-sach-quyen-rieng-tu
    


**PART 1: DATA PREPARATION**

Ở đây, ta sẽ xây dựng một lớp để xử lý việc tải và xử lý dữ liệu từ VNSTOCK3

In [ ]:
def load_data(symbol, date_start, date_end):
    """Loads data from vnstock3 for a given symbol.

    Args:
      symbol: The stock symbol (e.g., 'TCB', 'VIC').
      date_start: starting query date: 'YYYY-MM-DD'.
      date_end: ending query date: 'YYYY-MM-DD'.

    Returns:
      A pandas DataFrame containing the historical data for the given symbol,
      or None if an error occurs.
    """
    try:
        # Replace with actual vnstock3 data loading logic
        # Example using a placeholder CSV file
        # df = pd.read_csv(f"path/to/data/{symbol}.csv")
        stock = Vnstock().stock(symbol=symbol, source=symbol)
        return stock.quote.history(start=date_start, end=date_end)
    except FileNotFoundError:
        print(f"Error: Data for symbol '{symbol}' not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


**PART 2: ENVIROMENT**

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import matplotlib.pyplot as plt
class TradingEnv(gym.Env):
    def __init__(self, window_size,symbol):
        super(TradingEnv, self).__init__()
        self.data = load_data(symbol)
        self.window_size = window_size
        self.current_step = self.window_size
        self.action_space = spaces.Discrete(3)  # [0: Sell, 1: Hold, 2: Buy]
        self.observation_space = spaces.Box(
            low=0, high=1, shape=(self.window_size,), dtype=np.float32
        )
        self.inventory = []
        self.done = False
        self.logs = {'states_buy': [], 'states_sell': []}
        self.total_profit = 0

    def reset(self):
        self.current_step = self.window_size
        self.inventory = []
        self.done = False

        return self._next_observation()

    def _next_observation(self):
        if self.current_step < self.window_size:
            delta = self.window_size - self.current_step
            obs = np.concatenate(([0]*delta, self.data[:self.current_step]))
        else:
            obs = self.data[self.current_step - self.window_size:self.current_step]

        normalized_obs = self._normalize(obs)
        return normalized_obs

    def _normalize(self, data):
        return 1 / (1 + np.exp(-data))  # Sigmoid normalization

    def step(self, action):
        reward = 0
        prev_price = self.data[self.current_step - 1]
        current_price = self.data[self.current_step]

        # Action logic
        if action == 0:  # Sell
            reward = self.inventory.pop() - current_price
        elif action == 2:  # Buy
            self.inventory.append(current_price)
            reward = 0
        # Hold action doesn't change inventory or immediate reward

        self.current_step += 1

        if self.current_step >= len(self.data):
            self.done = True

        observation = self._next_observation()
        info = {}

        return observation, reward, self.done, info

    def render(self, mode='rgb'):
        plt.figure(figsize=(12, 6))
        plt.plot(self.data, color='r', lw=2.)
        plt.plot(self.data, 'v', markersize=10, color='m', label='Selling signal', markevery=self.logs["states_sell"])
        plt.plot(self.data, "^", markersize=10, color="k", label="Buy signal", markevery = self.logs["states_buy"])
        plt.title('Total gains: %f'%self.total_profit)
        plt.legend()
        plt.show()



In [ ]:
from gymnasium.envs.registration import register
register(
    id='NumberGuess-v0',
    entry_point=lambda: TradingEnv(max_number=20),  # Using lambda for parameters
)

In [ ]:
from stable_baselines3 import A2C

env = TradingEnv(window_size=10)

model = A2C('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=10000)
